In [5]:
import numpy as np

In [6]:
# Define constants 
Ts = 0.01
gravity = 9.81

dt = Ts

In [7]:
def q2dcm(q):
    
    qx = q[0]
    qy = q[1]
    qz = q[2]
    qw = q[3]
    
    DCM = np.array([
                     [1- 2*qz*qz- 2*qy*qy,- 2* qz*qw+2*qy*qx,2*qy*qw +2* qz*qx],
                     [2*qx*qy+ 2*qw*qz,1 - 2*qz*qz - 2*qx*qx,2*qz*qy- 2*qx*qw],
                     [2*qx*qz- 2*qw*qy,2*qy*qz + 2*qw*qx,1- 2*qy*qy- 2*qx*qx]
                    ])
    DCM = DCM.reshape(3,3)
    return DCM

#Test for a rotation about z of 90°
q_test = np.array([0.707, 0.707,0,0]).reshape(4,1)
v_test = np.array([1,0,0]).reshape(3,1);

dcm_test = q2dcm(q_test)
print(dcm_test)
v_test_rot = np.dot(dcm_test , v_test)

print(v_test_rot,v_test_rot.shape, np.linalg.norm(v_test_rot))

[[ 3.02000e-04  9.99698e-01  0.00000e+00]
 [ 9.99698e-01  3.02000e-04  0.00000e+00]
 [ 0.00000e+00  0.00000e+00 -9.99396e-01]]
(array([[3.02000e-04],
       [9.99698e-01],
       [0.00000e+00]]), (3, 1), 0.9996980456157748)


state  => x </br>

x = {px, py, pz,vx, vy, vz, qx, qy, qz, qw}^T   ==> size(x) = 10 x 1


control => u </br>

u = {Fx, Fy, Fz, Tx,Ty,Tz}^T

In [8]:
def gravity(x):
    #
    g_0 = np.array([0,0,-9.81]).reshape(3,1)
    return g_0

def navEq(x,u,dt):
    ### traslation
    pv = x[:6]
    q = x[6:]
    A = np.eye(6)
    
    A[0,3] = dt
    A[1,4] = dt
    A[2,5] = dt #here implements simple integrator: x[t] = x[t-1] + vx * dt
    
    f_t = np.dot(q2dcm(q),u[:3])
    acc_t = f_t - gravity(x)
    
    B = np.concatenate((np.eye(3,3) * dt**2/2,np.eye(3,3) * dt))
    
    x_pv = np.dot(A,x[:6])+ np.dot(B,f_t)
    
    ### rotation
    w_t = u[3:]
    P = w_t[0] * dt
    Q = w_t[1] * dt
    R = w_t[2] * dt
    
    
    OMEGA = np.array([[0, R, -Q, P],
                      [-R, 0, P, Q],
                      [Q, -P, 0, R],
                      [-P, -Q, -R, 0]])
    
    V = np.linalg.norm(w_t) * dt
    
    x_rot = np.dot(np.cos(V/2) * np.eye(4) + 2/V * np.sin(V/2) * OMEGA ,x[6:])
    
    x = np.zeros((10,1))
    x[:6] = x_pv
    x[6:] = x_rot
    
    return x

### Test
x_test = np.array([1,1,0,0.1,0.1,0,q_test[0],q_test[1],q_test[2],q_test[3]]).reshape(10,1)
u_test = np.array([1,0,0,1,2,0]).reshape(6,1)
x_test_t = navEq(x_test,u_test,dt)
print(x_test_t)

[[ 1.00100002]
 [ 1.00104998]
 [ 0.        ]
 [ 0.10000302]
 [ 0.10999698]
 [ 0.        ]
 [ 0.70695581]
 [ 0.70695581]
 [ 0.00706985]
 [-0.02120956]]
